In [9]:
import math
import numpy as np
import matplotlib  as plt

In [10]:
class Value:
    def __init__(self,data,_child=(),_op='',):
        self.data=data
        self.prev=set(_child)
        self._back=lambda:None
        self.grad=0.0
        self._op=_op

    def __repr__(self) :
        return f"Value(data={self.data}"  
      
    def __add__(self,other):
        out=Value(self.data + other.data,(self,other),'+')

        def _back():
            self.grad+=1.0*out.grad
            other.grad+=1.0*out.grad
            out._back=_back 

        return out
    
    def __mul__(self,other):
        out=Value(self.data * other.data,(self,other),'*')

        def _back():
            self.grad+=other.grad*out.grad
            other.grad+=self.grad*out.grad
            out._back=_back
        return out
    
    def tanh(self):
        x=self.data
        t=((math.exp(2*x)-1)/math.exp(2*x)+1)
        out=Value(t,(self,),'tanh')

        def _back():
            self.grad=(1-t**2)*out.grad
        out._back=_back    
        return out
    def back(self):
    

        topo=[]
        visited =set()
        def build_topo(v):
            visited.add(v)
            for child in v.prev:
                build_topo(child)
            topo.append(v)
        build_topo(self)
        topo        
        o.grad=1.0

        for node in reversed (topo):
            node._back()


In [11]:
x1=Value(2.0)
x2=Value(0.0)
w1=Value(-3.0)
w2=Value(1.0)
b=Value(6)
x1w1=x1*w1
x2w2=x2*w2
x1w1x2w2=x1w1*x2w2
n=x1w1x2w2
o=n.tanh();